# Copyright 2022 Cognite AS

## Import the Libraries and Modules

In [1]:
import sys
from pathlib import Path

utils = str(Path("../utils").resolve())
if utils not in sys.path:
    sys.path.append(utils)

from cognite_auth import interactive_client

## Create the Cognite Client

In [2]:
c = client = interactive_client()

## Delete the data in CDF

### Delete using ids

In [12]:
#Please use the same prefix as in the 3_Create_Update_insert notebook. If it is already defined, it's all good! If not, remove "#" from the line below and use the same prefix as when you created the data

prefix = "My_Prefix"

In [13]:
c.sequences.data.retrieve_dataframe(external_id=prefix+"_test_sequence",start=0,end=None)

,user,amount


Deleting a row from the sequence table

In [9]:
c.sequences.data.delete(external_id=prefix+"_test_sequence", rows=[2])

Looking at the result after the delete

In [14]:
c.sequences.data.retrieve_dataframe(external_id=prefix+"_test_sequence",start=0,end=None)

,user,amount


### Delete using a Range

In [15]:
from datetime import datetime

c.time_series.data.retrieve(start=(datetime(2020,1,1)), end=(datetime(2021,1,1)), external_id="timeseries_"+prefix)

,timeseries_My_Prefix
2019-12-31 18:30:00,1000.0
2020-01-01 00:00:00,800.0
2020-01-01 18:30:00,2000.0
2020-02-01 00:00:00,500.0


Deleting data in a specific timerange

In [16]:
from datetime import datetime

c.time_series.data.delete_range(start=(datetime(2020,1,1)), end=(datetime(2020,2,2)), external_id="timeseries_"+prefix)

Looking at the data we have left after the deletion 

In [17]:
from datetime import datetime

c.time_series.data.retrieve(start=(datetime(2020,1,1)), end=(datetime(2021,1,1)), external_id="timeseries_"+prefix)

,timeseries_My_Prefix


### Delete various resource types

In [18]:
c.labels.delete(external_id=["ROTATING_EQUIPMENT_"+prefix, "PUMP_"+prefix])

In [19]:
c.assets.delete(external_id="asset1_"+prefix)

In [20]:
c.time_series.delete(external_id="timeseries_"+prefix)

## Bulk Deletion

How to delete recursively and effectively.

In [21]:
c.assets.retrieve_subtree(external_id=prefix+"_root")

,external_id,name,data_set_id,metadata,id,created_time,last_updated_time,root_id,parent_id,parent_external_id
0,My_Prefix_root,My_Prefix Root,7954497947800251,{},7426575356979301,2024-09-16 11:41:40.329,2024-09-16 11:41:40.329,7426575356979301,<NA>,NaN
1,My_Prefix_child1,child1,7954497947800251,{},612024077343850,2024-09-16 11:41:40.329,2024-09-16 11:41:40.329,7426575356979301,7426575356979301,My_Prefix_root
2,My_Prefix_child2,child2,7954497947800251,{},6402720148303080,2024-09-16 11:41:40.329,2024-09-16 11:41:40.329,7426575356979301,7426575356979301,My_Prefix_root
3,My_Prefix_child_of_child1,child_of_child1,7954497947800251,{},7301090258208056,2024-09-16 11:41:40.329,2024-09-16 11:41:40.329,7426575356979301,612024077343850,My_Prefix_child1


Deleting the root asset and all the assets under the specific root asset.

In [22]:
# Delete root and all subtree assets:
c.assets.delete(external_id=prefix+"_root", recursive=True)

We can now look at the assets in our data set and see that it is empty

In [23]:
c.assets.list(data_set_external_ids=[prefix])

,name,data_set_id,metadata,labels,id,created_time,last_updated_time,root_id,external_id
0,My_Prefix_test_pump,7954497947800251,{},[{'external_id': 'PUMP_My_Prefix'}],888554096935235,2024-09-16 11:41:30.521,2024-09-16 11:41:30.521,888554096935235,NaN
1,asset2,7954497947800251,{},NaN,5551029858893012,2024-09-16 11:39:56.765,2024-09-16 11:39:56.765,5551029858893012,asset2_My_Prefix


If the data set is not empty, we can run this code to delete the remaining assets

In [24]:
assets_ids=[]

for asset in c.assets.list(data_set_external_ids=[prefix], limit=-1):
    assets_ids.append(asset.id)

c.assets.delete(id=assets_ids)